In [ ]:
from mcstasscript.interface import instr, plotter, functions

# Creating the instance of the class, insert path to mcrun and to mcstas root directory
detector = instr.McStas_instr("LOKI_detector",
                              mcrun_path = "/Applications/McStas-2.5.app/Contents/Resources/mcstas/2.5/bin",
                              mcstas_path = "/Applications/McStas-2.5.app/Contents/Resources/mcstas/2.5")

In [ ]:
detector.add_parameter("wavelength")
detector.add_parameter("height",value=1.0,comment="Height in [m]")
detector.add_parameter("string","reflection_filename")
detector.add_parameter("string","data_filename",value="\"data.dat\"")

In [ ]:
detector.add_declare_var("double","energy")
detector.add_declare_var("int","flag")
detector.add_declare_var("double","tube_radius",value=0.013)
detector.add_declare_var("double","displacements",array=7)
detector.add_declare_var("double","t_array",array=4,value=[0.65E-6,0.65E-6,1E-6,1E-6])

In [ ]:
detector.append_initialize("energy=pow(2*PI/wavelength*K2V,2)*VS2E;")

In [ ]:
detector.add_component("Origin","Arm")
src = detector.add_component("source","Source_simple",RELATIVE="Origin")
detector.add_component("beam_extraction","Guide_gravity",AT=[0,0,2],RELATIVE="source")

In [ ]:
detector.add_component("pre_guide_slit","Slit",before="beam_extraction",
                       AT=[0,0,1],RELATIVE="source",comment="Slit before the guide")

In [ ]:
detector.print_components()

In [ ]:
detector.set_component_parameter("source",{"xwidth" : 0.12, "E0" : "energy"})
detector.set_component_parameter("source",{"yheight" : 0.12})

In [ ]:
detector.print_component("source")

In [ ]:
detector.set_component_AT("source",[0.01,0,0])

In [ ]:
detector.set_component_ROTATED("beam_extraction",[0,2.0,0],RELATIVE="Origin")

In [ ]:
detector.set_component_RELATIVE("beam_extraction","pre_guide_slit")

In [ ]:
detector.print_components()

In [ ]:
detector.set_component_WHEN("beam_extraction","vx > 0")
detector.print_component("beam_extraction")

In [ ]:
detector.append_component_EXTEND("beam_extraction","n_scattering = SCATTERED - 2")
detector.print_component("beam_extraction")

In [ ]:
detector.set_component_GROUP("beam_extraction","guides")
detector.print_component("beam_extraction")

In [ ]:
detector.set_component_JUMP("beam_extraction","myself iterate 3")
detector.print_component("beam_extraction")

In [ ]:
detector.set_component_comment("beam_extraction","Simulating severe misalignment")
detector.print_component("beam_extraction")

In [ ]:
detector.show_components()

In [ ]:
detector.show_components("samples")

In [ ]:
detector.component_help("Phonon_simple")